In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pickle
import numpy as np

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        
        return x

In [3]:
def train(model, x_train, y_train, epochs=5):
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    train_loss_list = []
    for epoch in range(epochs):  
        
        running_loss = 0.0
        for inputs, targets in zip(x_train, y_train):
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        train_loss_list.append(running_loss)
        
        print('Train Epoch: %i, Loss: %f' % (epoch+1, running_loss))
    print('Finished Training')
    return train_loss_list

In [4]:
def dataloader():
    
    with open('param.pkl','rb') as f:
        data = pickle.load(f)
    inputs = torch.tensor(data['inputs']).float()
    w1 = data["w1"]
    w2 = data["w2"]
    w3 = data["w3"]
    b1 = data["b1"]
    b2 = data["b2"] 
    b3 = data["b3"]
    targets = torch.tensor(data['targets']).float()
    params = [
        (w1, b1),
        (w2, b2),
        (w3, b3)
    ]
    trainset = (inputs, targets)

    return trainset, params


In [5]:
trainset, params = dataloader()
model = MLP()

model.fc1.weight.data = torch.tensor(params[0][0]).float()
model.fc1.bias.data = torch.tensor(params[0][1]).float()
model.fc2.weight.data = torch.tensor(params[1][0]).float()
model.fc2.bias.data = torch.tensor(params[1][1]).float()
model.fc3.weight.data = torch.tensor(params[2][0]).float()
model.fc3.bias.data = torch.tensor(params[2][1]).float()
train(model, trainset[0], trainset[1])

/Users/liyang/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Train Epoch: 1, Loss: 133.454646
Train Epoch: 2, Loss: 100.823284
Train Epoch: 3, Loss: 98.950021
Train Epoch: 4, Loss: 97.195962
Train Epoch: 5, Loss: 96.158900
Finished Training


[133.45464649924543,
 100.8232843612742,
 98.95002082383508,
 97.19596192679091,
 96.15890031944195]

In [6]:
from torchsummary import summary
summary(model,(200,2),batch_size=1,device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [1, 200, 10]              30
              ReLU-2               [1, 200, 10]               0
            Linear-3               [1, 200, 10]             110
              ReLU-4               [1, 200, 10]               0
            Linear-5                [1, 200, 1]              11
           Sigmoid-6                [1, 200, 1]               0
Total params: 151
Trainable params: 151
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.00
Estimated Total Size (MB): 0.07
----------------------------------------------------------------


torch.Size([200])